In [117]:
#!pip install sqlalchemy==1.3.9
#!pip install ibm_db_sa
#!pip install ipython-sql
import pandas
import datetime

In [74]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [75]:
#Create database connection

%sql ibm_db_sa://blg11630:jsnv%2Bw47652s064m@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB

DB2/LINUXX8664


In [4]:
#Showing all the tables in the database

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
BLG11630,CENSUS_DATA,2021-05-31 20:48:27.134126
BLG11630,CHICAGO_PUBLIC_SCHOOLS,2021-05-31 20:50:10.198124
BLG11630,CHICAGO_CRIME_DATA,2021-05-31 20:51:23.002158
BLG11630,US_FOMC_RATES,2021-06-02 14:51:56.612101
BLG11630,US_ISM_MANUFACTURING_PMI,2021-06-02 14:53:46.824188
BLG11630,US_CPI,2021-06-02 14:49:53.314324
BLG11630,US_ISM_SERVICES_PMI,2021-06-02 14:54:59.504757
BLG11630,US_NFP_RATES,2021-06-02 14:55:56.584638
BLG11630,US_TRADE_BALANCE,2021-06-02 14:57:34.241315
BLG11630,US_RETAIL_SALES,2021-06-02 14:58:36.464754


In [5]:
#Showing the first five rows of TEST_CPI table after converting to Pandas Dataframe
#TEST_CPI is a table currently used ONLY for the purpose of testing the codes
#Disregard it for the Forex Factory project.
#Also Disregard Census_data, Chicago_public_schools and Chicago_Crime_Data Tables for the purpose of the forex factory project.

pd = %sql select * from TEST_CPI
pd.DataFrame().head()

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


,DATE,VALUE
0,2021-05-12,0.008
1,2021-04-13,0.006
2,2021-03-10,0.004
3,2021-02-10,0.003
4,2021-01-13,0.004


In [96]:
#Updating the database

%sql INSERT INTO TEST_CPI VALUES ('2021-06-03', 0.005), ('2021-06-01', 0.007)

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
2 rows affected.


[]

In [97]:
#showing the result of the updated database
result = %sql Select * from TEST_CPI
result.DataFrame()

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


,DATE,VALUE
0,2021-06-03,0.005
1,2021-06-01,0.007
2,2021-06-03,0.005
3,2021-06-01,0.007
4,2021-05-12,0.008
5,2021-04-13,0.006
6,2021-03-10,0.004
7,2021-02-10,0.003
8,2021-01-13,0.004
9,2020-12-10,0.002


In [19]:
#Demonstrating how to add from pandas dataframe to database using variables
hello = {'Title': 'Final Manufacturing PMI',
  'Date': datetime.datetime(2021, 5, 3, 0, 0),
  'Actual': '60.5'}, {'Title': 'ISM Manufacturing PMI',
  'Date': datetime.datetime(2021, 5, 3, 0, 0),
  'Actual': '60.7'}, {'Title': 'Trade Balance',
  'Date': datetime.datetime(2021, 5, 4, 0, 0),
  'Actual': '-74.4B'}

test = pandas.DataFrame(hello)

In [119]:
test_trade_balance = test.loc[test.Title == 'Trade Balance']
tb_date = test_trade_balance['Date']
tb_value = test_trade_balance['Actual']

#removing all the unwanted symbols
test_value = float(str(list(tb_value)).strip('B[]\''))
test_date = str(list(tb_date)).strip('Timestamp([])\'').split(' ')[0]

In [120]:
#Adding from pandas dataframe to database using variables

%sql INSERT INTO TEST_CPI VALUES (:test_date, :test_value)

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
1 rows affected.


[]

In [121]:
#showing updated results

result = %sql Select * from TEST_CPI
result.DataFrame()

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


,DATE,VALUE
0,2021-06-03,0.005
1,2021-06-01,0.007
2,2021-06-03,0.005
3,2021-06-01,0.007
4,2021-05-04,-74.400
5,2021-05-04,-74.400
6,2021-05-04,-74.400
7,2021-05-04,-74.400
8,2021-05-04,-74.400
9,2021-05-12,0.008
